In [1]:
from PreProcessor import *
from DatasetRetrieval import *
_PreProcessor = PreProcessor()

c:\ProgramData\anaconda3\envs\cude_env2\lib\site-packages\tree_sitter\__init__.py:36: FutureWarning: Language.build_library is deprecated. Use the new bindings instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)
c:\ProgramData\anaconda3\envs\cude_env2\lib\site-packages\tree_sitter\__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


In [2]:
_dataset_retrieval_obj = DatasetRetrieval()
merged_df = _dataset_retrieval_obj.parseSqlite("Dataset/errai_dataset/errai.sqlite3")
CC_UC_dict_true, UC_to_index, index_to_UC, CC_index_dict, index_to_CC = _dataset_retrieval_obj.getGlobalDatasetData(merged_df, 'errai_repo_clone')
CC_train_docs, UC_train_docs, CC_test_docs, UC_test_docs, CC_UC_train, CC_UC_test, CC_UC_train_labels, CC_UC_test_labels = _dataset_retrieval_obj.splitTestTrain(CC_UC_dict_true, UC_to_index, index_to_UC, CC_index_dict, index_to_CC)

In [3]:
print(len(CC_train_docs))

2050


In [3]:
CC_train = _PreProcessor.setupDeepLearning(CC_train_docs,'CC','train')
UC_train = _PreProcessor.setupDeepLearning(UC_train_docs,'UC','train')
CC_test = _PreProcessor.setupDeepLearning(CC_test_docs,'CC','test')
UC_test = _PreProcessor.setupDeepLearning(UC_test_docs,'UC','test')

In [ ]:
print(len(CC_train_docs))

In [4]:
_PreProcessor.setUpUnknown(CC_train,'train')
_PreProcessor.setUpUnknown(UC_train,'train')

_PreProcessor.setUpUnknown(CC_test,'test')
_PreProcessor.setUpUnknown(UC_test,'test')

In [5]:
with open('./pickles/DeepLearning/CC_train.pkl', 'wb') as f:
       pickle.dump(CC_train, f)

with open('./pickles/DeepLearning/UC_train.pkl', 'wb') as f:
        pickle.dump(UC_train, f)

with open('./pickles/DeepLearning/CC_test.pkl', 'wb') as f:
       pickle.dump(CC_test, f)
with open('./pickles/DeepLearning/UC_test.pkl', 'wb') as f:
        pickle.dump(UC_test, f)


In [6]:
CC_train = np.load('./pickles/DeepLearning/CC_train.pkl',allow_pickle=True)
UC_train = np.load('./pickles/DeepLearning/UC_train.pkl',allow_pickle=True)
CC_test = np.load('./pickles/DeepLearning/CC_test.pkl',allow_pickle=True)
UC_test = np.load('./pickles/DeepLearning/UC_test.pkl',allow_pickle=True)

In [ ]:
print(CC_test[0:10])

In [7]:
# CC_word2vec = _PreProcessor.word2VecProcessor(function_names_train, function_segments_train, 'CC')
# UC_word2vec = _PreProcessor.word2VecProcessor(summaries_train, descriptions_train, 'UC')
# print(CC_word2vec)
word2vec_model = Word2Vec(sentences = CC_train + UC_train, vector_size=200, window=5, min_count=1, workers=4, epochs=10)
word2vec_model.save("./Dataset/teiid_dataset/UC_CC_WORD2VEC")

#word2vec_model.build_vocab(UC_word2vec, update=True)
#word2vec_model.train(UC_word2vec, total_examples=word2vec_model.corpus_count, epochs=word2vec_model.epochs)

# word2vec_model.save("./Dataset/teiid_dataset/UC_CC_WORD2VEC")
word2vec_model = Word2Vec.load("./Dataset/teiid_dataset/UC_CC_WORD2VEC")

word2vec_vocab = word2vec_model.wv.index_to_key

# Initialize an empty embedding matrix
embedding_matrix = np.zeros((len(word2vec_vocab) + 1, word2vec_model.vector_size))

# Fill the embedding matrix with the embeddings of each word
for i, word in enumerate(word2vec_vocab):
    embedding_vector = word2vec_model.wv[word]
    if embedding_vector is not None:
        embedding_matrix[i + 1] = embedding_vector

with open('./pickles/DeepLearning/embedding_matrix.pkl', 'wb') as f:
       pickle.dump(embedding_matrix, f)



In [8]:
embedding_matrix = np.load('./pickles/DeepLearning/embedding_matrix.pkl',allow_pickle=True)
print(len(embedding_matrix))

3590


In [ ]:
print(UC_test[:2])

### Converting dataset to indices

In [9]:
_PreProcessor.vocabToIndex(word2vec_vocab)
_PreProcessor.dataSetToIndex(CC_train)
_PreProcessor.dataSetToIndex(UC_train)

_PreProcessor.dataSetToIndex(CC_test)
_PreProcessor.dataSetToIndex(UC_test)

In [ ]:
print(len(UC_train[0]))

In [ ]:
# min_len = 100000000
# #cc => 4000
# print(len(UC_train))
# import statistics
# data = []

# for cc in UC_train:
#     data.append(len(cc))
# # Sample data

# # Calculate the mean
# mean_value = statistics.mean(data)
# print(mean_value)

# counter=0
# for cc in UC_train:
#     if len(cc) > mean_value + 800:
#         counter+=1
# print(counter)
print(len(CC_train))

In [10]:
x_train, labels = _PreProcessor.setUpLabels(CC_train,UC_train,"dataset/errai_dataset/train.csv")

In [11]:
x_test, labels_test = _PreProcessor.setUpLabels(CC_test,UC_test,"dataset/errai_dataset/test.csv")

In [ ]:
print(len(x_train))
print(len(labels))
print(len(x_test))
print(len(labels_test))

In [12]:
with open('./pickles/DeepLearning/x_train.pkl', 'wb') as f:
       pickle.dump(x_train, f)
with open('./pickles/DeepLearning/labels.pkl', 'wb') as f:
        pickle.dump(labels, f)

In [13]:
with open('./pickles/DeepLearning/x_test.pkl', 'wb') as f:
       pickle.dump(x_test, f)
with open('./pickles/DeepLearning/labels_test.pkl', 'wb') as f:
        pickle.dump(labels_test, f)

In [14]:
x_train = np.load('./pickles/DeepLearning/x_train.pkl',allow_pickle=True)
labels = np.load('./pickles/DeepLearning/labels.pkl',allow_pickle=True)


In [15]:
x_test = np.load('./pickles/DeepLearning/x_test.pkl',allow_pickle=True)
labels_test = np.load('./pickles/DeepLearning/labels_test.pkl',allow_pickle=True)

In [16]:
class TracibilityLinkDataset(Dataset):

    def __init__(self, x_train, y_train):
        
        self.tensor_x_train = x_train
        self.tensor_y_train = torch.tensor(y_train)


    # len and get item are very important --> used by dataloader
    def __len__(self):
        return len(self.tensor_y_train)

    def __getitem__(self, idx):
        return self.tensor_x_train[idx], self.tensor_y_train[idx]


In [17]:
dataset_train = TracibilityLinkDataset(x_train, labels)

In [18]:
dataset_test = TracibilityLinkDataset(x_test, labels_test)

In [ ]:
# torch.save(dataset_train,'./dataset/teiid_dataset/DeepLearningDataset.pt')


In [26]:
dataset_train = torch.load('/kaggle/input/dataset-200/dataset_train_200.pt')


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/dataset-200/dataset_train_200.pt'

In [ ]:
dataset_test = torch.load('/kaggle/input/dataset-200/dataset_test_200.pt')


In [17]:
import sklearn.utils.class_weight as scikit_class_weight

class_list = [0,1]
class_weight_value = scikit_class_weight.compute_class_weight(class_weight ='balanced', classes = class_list, y = dataset_train.tensor_y_train.tolist())
class_weight = dict()
print(len(class_weight_value)) #2

for i in range(2):
    class_weight[i] = 1
    
print(class_weight)
# Build the dictionary using the weight obtained the scikit function
for i in range(len(class_list)):
    class_weight[class_list[i]] = class_weight_value[i]

print(class_weight)

InvalidParameterError: The 'classes' parameter of compute_class_weight must be an instance of 'numpy.ndarray'. Got [0, 1] instead.

In [33]:
x_train_small = []
label_small = []
counter_1 = 0
counter_0 = 0
while(counter_1 < 11 or counter_0 < 11):
    index = random.randint(0, len(x_train))

    if labels[index] == 1 and counter_1 < 11 :
        x_train_small.append(x_train[index])
        label_small.append(1)
        counter_1 +=1

    elif labels[index] == 0 and counter_0 < 11 :
        x_train_small.append(x_train[index])
        label_small.append(0)
        counter_0 +=1
dataset_small = TracibilityLinkDataset(x_train_small, label_small)    

In [19]:
class DLModel(nn.Module):

  def __init__(self,embedding_matrix : np.array, embedding_dim : tuple,UC_size: int, CC_size:int, hidden_size:int = 64, classes: int = 2):
      super(DLModel, self).__init__()

 
      self.embedding = nn.Embedding(num_embeddings = embedding_dim[0], embedding_dim = embedding_dim[1], _weight = torch.tensor(embedding_matrix))
      
      self.lstm_CC = nn.LSTM(embedding_dim[1],hidden_size,batch_first=True, bidirectional=False)
        
      # self.linear_post_flatten = nn.Linear(CC_size*hidden_size+UC_size*hidden_size,128)
      # self.linear_post_flatten2 = nn.Linear(128,32)

      self.conv_layer = nn.Conv1d(CC_size+UC_size,hidden_size*2 ,3,stride=3)
      self.linear = nn.Linear(hidden_size*2*21,classes)

      self.softmax = nn.Softmax(dim=1)
      
      # self.cosine_similarity = nn.CosineSimilarity(dim=1) # code = 1*(vocab_size)*(feature_vector) , req = code = 1*1000*200 

      # self.sigmoid = nn.Sigmoid()
    
  def forward(self, CC,UC ):
    #function_names.shape --> batch_no * no_tokens* 150
    
    CC_embedding  = self.embedding(CC)
    UC_embedding = self.embedding(UC)
    
    #function_names.shape --> batch_no * no_tokens* 150
    
    
    UC_lstm,_ = self.lstm_CC (UC_embedding.float())
    CC_lstm,_ = self.lstm_CC (CC_embedding.float())

    
    #function_names.shape --> batch_no * no_tokens* 150

    conv = self.conv_layer (torch.cat([UC_lstm,CC_lstm],axis=1).float())
    conv_flatten = conv.reshape(conv.shape[0],-1)

    # UC_lstm_flatten = UC_lstm.reshape(UC_lstm.shape[0],-1)
    # CC_lstm_flatten = CC_lstm.reshape(CC_lstm.shape[0],-1) # batch_no * (feature_vector * no_tokens)


    # linear_output_one = self.linear_post_flatten(torch.cat([UC_lstm_flatten,CC_lstm_flatten],axis=1).float())
    # print(linear_output_one)
    # linear_output_two = self.linear_post_flatten2(linear_output_one.float())
    # print(conv.shape)
    linear_output = self.linear(conv_flatten)
    
    

    return linear_output


In [21]:
UC_size  = 1000
CC_size = 4000

In [20]:
from torch.nn.utils.rnn import pad_sequence

def weighted_binary_cross_entropy(output, target):
        
#     loss = weights[1] *(target * torch.log(output)) + \
#            weights[0] * ((1 - target) * torch.log(1 - output))
#     output --> kber ,loss 
    loss = (target * torch.log(output)) + \
            ((1 - target) * torch.log(1 - output))

    return torch.mean(loss)

def customCollate(batch: list):
    # batch --> tuple (x,y)
    # x --> list of 4 lists
    x_batch, y_batch = zip(*batch)
    
    CC,UC = zip(*x_batch)
    
    CC_padded = []
    UC_padded = []
    
    for i  in range(len(x_batch)):
        
        CC_padded.append( list(CC[i]) + [0] * (CC_size - len(CC[i])) )
        UC_padded.append( list(UC[i]) + [0] * (UC_size - len(UC[i])) )
        

    return  CC_padded,UC_padded, y_batch

def train(model, train_dataset, batch_size=50, epochs=1 ,learning_rate=0.001):
    
    # (1) create the dataloader of the training set IMPORTANT (make the shuffle=True)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True ,collate_fn = customCollate)
    
    
    # (2) make the criterion cross entropy loss
    criterion = torch.nn.CrossEntropyLoss()
    # (3) create the optimizer (Adam)
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu") 

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()
   

    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0
        n_samples = 0
        y_size = 0
        true_pos = 0
        false_neg = 0
        false_pos = 0 
        true_neg = 0
        for CC,UC, y_batch in tqdm(train_dataloader):
            # torch.cuda.empty_cache()     
            CC_tensor =torch.tensor(CC, device=device)
            UC_tensor = torch.tensor(UC, device=device)
           
            y_batch_tensor = torch.tensor(y_batch,dtype = torch.float32 ,device=device)
            
            output = model.forward(CC_tensor,UC_tensor)
#             _,predicted=torch.max(output)  #512 * 104  for every word in every sentence we choose one tag form the seventen tag 
#             output_0_1 = torch.tensor((output > 0.5), dtype=torch.float32)
            
            # for i,value in enumerate(output_0_1): #calc the fals negtive
            #     if value == 0 and  y_batch_tensor[i] == 1:
            #       false_neg+=1
            #     elif value == 1 and  y_batch_tensor[i] == 1:
            #       true_pos+=1
            #     elif value == 1 and  y_batch_tensor[i] == 0:
            #       false_pos += 1
            #     else:
            #       true_neg += 1
            
#             weights = torch.where(y_batch_tensor == 0, torch.tensor(class_weights[0]).to(device), torch.tensor(class_weights[1]).to(device)).to(device)
            
            _,predicted=torch.max(output,dim=1)
            batch_loss = criterion(output,y_batch_tensor.long())
            # batch_loss = criterion(output, y_batch_tensor)
            # torch.cuda.empty_cache()
            total_loss_train += batch_loss
            # print(torch.cuda.memory_reserved()/1024**3)
              

            # (7) loss calculation (you need to think in this part how to calculate the loss correctly)
            
    #       (9) calculate the batch accuracy (just add the number of correct predictions)
            acc = (predicted==y_batch_tensor).sum().item()
            n_samples += y_batch_tensor.size(0)
            total_acc_train += acc


            optimizer.zero_grad()

    #       (11) do the backward pass
            batch_loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    #       (12) update the weights with your optimizer
            optimizer.step()
            # (10) zero your gradients
            
    #   epoch loss
        epoch_loss = total_loss_train / n_samples

    #   (13) calculate the accuracy
        epoch_acc =100*total_acc_train/n_samples

        print(
           f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
           | Train Accuracy: {epoch_acc}\n')
        print("true_pos = ", true_pos)
        print("false_neg = ",false_neg)
        print("false_pos = ",false_pos)
        print("true_neg = ", true_neg)
        

  ##############################################################################################################

In [22]:
model = DLModel(embedding_matrix, embedding_matrix.shape,UC_size,CC_size, hidden_size=64, classes=2)    
# model.load_state_dict(torch.load('ModelCrossentropyLoss_lstmLayer+linear_'+str(i-1)+'_epoch_weight24_0.001.pth'))
# model.train()
train(model, dataset_train)
# torch.save(model.state_dict(), 'ModelCrossentropyLoss_lstmLayer+linear_'+str(i)+'_epoch_weight24_0.001.pth')

100%|██████████| 86/86 [06:57<00:00,  4.85s/it]

Epochs: 1 | Train Loss: 0.01494288444519043            | Train Accuracy: 51.951104842501174

true_pos =  0
false_neg =  0
false_pos =  0
true_neg =  0


In [ ]:
torch.save(model.state_dict(), 'SelfAttentionModel_?epochs_200_tokens.pth')

In [ ]:
epoch_loss = total_loss_train / torch.tensor(y_batch).size(0)

    #   (13) calculate the accuracy
epoch_acc =100*total_acc_train/n_samples

In [ ]:
model = DLModel(embedding_matrix, embedding_matrix.shape)
model.load_state_dict(torch.load('/kaggle/working/SelfAttentionModel_?epochs_200_tokens.pth'))
model.train()

In [23]:
def customCollatetest(batch: list):
    # batch --> tuple (x,y)
    # x --> list of 4 lists
    x_batch, y_batch = zip(*batch)
    
    CC,UC = zip(*x_batch)
    
    CC_padded = []
    UC_padded = []
    
    for i  in range(len(x_batch)):
        
        CC_padded.append( list(CC[i]) + [0] * (CC_size - len(CC[i])) )
        UC_padded.append( list(UC[i]) + [0] * (UC_size - len(UC[i])) )
        

    return  CC_padded,UC_padded, y_batch
    

def evaluate(model, test_dataset,batch_size=5):
  """
  This function takes a NER model and evaluates its performance (accuracy) on a test data
  Inputs:
  - model: a NER model
  - test_dataset: dataset of type NERDataset
  """
  ########################### TODO: Replace the Nones in the following code ##########################

  # (1) create the test data loader
  test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, collate_fn=customCollatetest , shuffle=False)

  total_acc_test = 0
  n_samples = 0

  use_cuda = torch.cuda.is_available()
    
  device = torch.device("cuda" if use_cuda else "cpu") 
    # device="cpu"
  if use_cuda:
        model = model.cuda()
  with torch.no_grad():
    false_neg = 0
    true_pos = 0
    false_pos = 0
    true_neg = 0
    for  CC,UC, y_batch in tqdm(test_dataloader):
        
            UC_padded_tensor = torch.tensor(UC, device=device)
            CC_padded_tensor = torch.tensor(CC, device=device)

            y_batch_tensor = torch.tensor(y_batch,dtype = torch.float32,device=device)
#             y_batch_tensor = torch.where(y_batch_tensor == 0 , torch.tensor(-1).to(device), torch.tensor(1).to(device)).to(device)
            output = model.forward(CC_padded_tensor,UC_padded_tensor)
#             print(output)
            _,predicted=torch.max(output,dim=1)

            for i,value in enumerate(predicted): #calc the fals negtive
                if value == 0 and  y_batch_tensor[i] == 1:
                  false_neg+=1
                elif value == 1 and  y_batch_tensor[i] == 1:
                  true_pos+=1
                elif value == 1 and  y_batch_tensor[i] == 0:
                  false_pos += 1
                else:
                  true_neg += 1
                
            n_true = (predicted==y_batch_tensor).sum().item()

            n_samples += y_batch_tensor.size(0)
            total_acc_test += n_true
       
    # (6) calculate the over all accuracy
    acc_test =100*total_acc_test/n_samples
  ##################################################################################################
    print("true_pos = ", true_pos)
    print("false_neg = ",false_neg)
    print("false_pos = ",false_pos)
    print("true_neg = ", true_neg)

    recall = true_pos / (true_pos+false_neg)
    precesion =  true_pos / (true_pos+false_pos)
    f1Score = (2*precesion*recall)/(precesion+recall)
    print(f'\nTest Accuracy: {acc_test}')
    print(f'\nTest Recall: {recall}') 
    print(f'\nTest precesion: {precesion}') 
    print(f'\nF1 Score: {f1Score}')

In [24]:
model.eval()
evaluate(model, dataset_test)

100%|██████████| 272/272 [01:13<00:00,  3.71it/s]

true_pos =  261
false_neg =  417
false_pos =  112
true_neg =  566

Test Accuracy: 60.9882005899705

Test Recall: 0.38495575221238937

Test precesion: 0.6997319034852547
